In [11]:
%pip install numpy transformers torch faiss-cpu

import numpy as np
from transformers import AutoTokenizer, AutoModel
import torch
import faiss

# Load pre-trained tokenizer and model
model_name = "sentence-transformers/all-MiniLM-L6-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Text to chunk and embed
text = """Artificial Intelligence (AI) is a rapidly growing field of technology that aims to create machines capable of intelligent behavior. 
Machine Learning (ML) is a subset of AI that involves training models on data to make predictions or decisions without being explicitly programmed. 
Natural Language Processing (NLP) is an area of AI that focuses on enabling machines to understand, interpret, and generate human language."""

# Chunk the text into sentences
chunks = text.split(". ")

# Create embeddings for each chunk
def embed_text(chunks):
    inputs = tokenizer(chunks, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state[:, 0, :]
    return embeddings

embeddings = embed_text(chunks)
embeddings_np = embeddings.numpy()

# Index the embeddings using FAISS
embedding_dim = embeddings_np.shape[1]
index = faiss.IndexFlatL2(embedding_dim)
index.add(embeddings_np)

# Function to find relevant chunk based on user query
def find_most_relevant_chunk(query, index, chunks):
    # Embed the query
    query_embedding = embed_text([query]).numpy()
    
    # Search the index
    _, indices = index.search(query_embedding, k=1)
    print("Indices:", indices,"chunks:",chunks)
    # Return the most relevant chunk
    return chunks[indices[0][0]]

# Example user query
query = "What is ML?"
most_relevant_chunk = find_most_relevant_chunk(query, index, chunks)
print("Most relevant chunk:", most_relevant_chunk)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.
Indices: [[1]] chunks: ['Artificial Intelligence (AI) is a rapidly growing field of technology that aims to create machines capable of intelligent behavior', '\nMachine Learning (ML) is a subset of AI that involves training models on data to make predictions or decisions without being explicitly programmed', '\nNatural Language Processing (NLP) is an area of AI that focuses on enabling machines to understand, interpret, and generate human language.']
Most relevant chunk: 
Machine Learning (ML) is a subset of AI that involves training models on data to make predictions or decisions without being explicitly programmed
